In [18]:

#!/usr/bin/env python3
"""
Jira에서 이슈들을 가져와서 JSONL 형태로 변환하는 스크립트 (임베딩용)
"""

import requests
import json
import time
import os
from datetime import datetime

# --- 설정 ---
JIRA_URL = "https://jira.suprema.co.kr"
JIRA_USERNAME = "dhwoo"
JIRA_PASSWORD = "android0237@"
JIRA_AUTH = (JIRA_USERNAME, JIRA_PASSWORD)
# 최종 결과가 저장될 폴더 이름
OUTPUT_FOLDER = "jira_issues_output" # 저장 폴더 이름을 더 명확하게 변경

# 검색할 JQL 쿼리
JQL_QUERY = 'project = "COMMONR" AND issuetype = "Test"'


def get_all_jira_issue_keys(jql):
    """JQL로 모든 이슈를 검색하여 이슈 키 리스트를 반환합니다."""
    all_issue_keys = []
    start_at = 0
    max_results = 100

    print(f"JQL로 이슈 검색을 시작합니다: {jql}")

    while True:
        url = f"{JIRA_URL}/rest/api/2/search"
        headers = {"Accept": "application/json"}
        params = {
            'jql': jql,
            'fields': 'key',
            'startAt': start_at,
            'maxResults': max_results
        }

        try:
            response = requests.get(url, headers=headers, params=params, auth=JIRA_AUTH)
            response.raise_for_status()
            data = response.json()
            issues_on_page = data.get('issues', [])
            
            if not issues_on_page:
                break
            
            keys_on_page = [issue['key'] for issue in issues_on_page]
            all_issue_keys.extend(keys_on_page)
            
            start_at += len(issues_on_page)
            print(f"  -> 현재까지 {len(all_issue_keys)} / {data['total']} 개 이슈 키를 가져왔습니다...")
            
        except requests.exceptions.RequestException as e:
            print(f"Jira 이슈 검색 오류: {e}")
            return None
            
    return all_issue_keys

In [19]:
def convert_to_embedding_format(issue_key):
    """하나의 이슈 키에 대한 모든 정보를 Jira API를 통해 가져옵니다."""
    url = f"{JIRA_URL}/rest/api/2/issue/{issue_key}"
    headers = {"Accept": "application/json"}
    
    try:
        response = requests.get(url, headers=headers, auth=JIRA_AUTH)
        response.raise_for_status()

        ''' 응답한 json을 issue_data로 가공합니다.'''
        issue_data = response.json()

        """이슈 데이터를 임베딩에 적합한 형태로 변환합니다."""
        if not issue_data:
            return None
        

        

        # 각 필드별로 리스트에서 모든 값들을 수집
        test_steps = []
        data_list = []
        expected_results = []

        #임베딩 리스트 생성
        embedding_list = []
        
        # customfield_10004에서 test step, data, expected result 추출 (리스트 형식)
        fields = issue_data.get('fields', {})
        description = fields.get('description', '')
        custom_field_10004 = fields.get('customfield_10004', {})
        steps_list = custom_field_10004.get('steps', [])
        
        # Issue Key
        issue_key = issue_data.get('key', '')

        
        if isinstance(steps_list, list):
            for item in steps_list:
                if isinstance(item, dict):
                    # 가능한 키 이름들을 시도
                    index = item.get('index', '')
                    step = item.get('step', '')
                    data_item = item.get('data', '')
                    expected_result = item.get('result','')
                    if step:
                        test_steps.append(step)
                    if data_item:
                        data_list.append(data_item)
                    if expected_result:
                        expected_results.append(expected_result)

                    #임베딩 리스트에 추가
                    embedding_list.append([index,step,data_item,expected_result])
        
        
        # JSONL용 객체 생성
        jsonl_object = {
            "id": issue_key,
            "content": embedding_list,
            "metadata": {
                "key": issue_key,
                "description": description,
                "test_step": test_steps,
                "data": data_list,
                "expected_result": expected_results,
            }
        }

    except requests.exceptions.RequestException as e:
        print(f"'{issue_key}'의 정보 조회 중 오류 발생: {e}")
        return None


    return jsonl_object

In [20]:
def json_to_save_file(json_data,issue_key) : 
    #1. 결과를 저장할 폴더 생성
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    print(f" {issue_key} 이슈의 정보를 json으로 저장을 시작합니다.")
    print(f"저장 폴더: '{os.path.abspath(OUTPUT_FOLDER)}'")

    #2. json 파일로 저장장
    try:
        output_filepath = os.path.join(OUTPUT_FOLDER, f"{issue_key}.json")
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=2)
        return True
    except IOError as e:
        print(f"'{output_filepath}' 파일 저장 중 오류 발생: {e}")
        return False

In [21]:
def main():
    """메인 실행 함수"""
    print("=== Jira to JSONL 변환 (임베딩용) ===")
    print(f"Jira URL: {JIRA_URL}")
    print(f"사용자: {JIRA_USERNAME}")
    print(f"JQL 쿼리: {JQL_QUERY}")
    print()

    # 1. 이슈 키 목록 가져오기
    issue_keys = get_all_jira_issue_keys(JQL_QUERY)
    
    if not issue_keys:
        print("검색된 이슈가 없습니다.")
        return
    
    print(f"총 {len(issue_keys)}개의 이슈를 찾았습니다.")
    print()
    
    # 2. 모든 이슈 데이터를 JSONL 형태로 변환
    jsonl_data = []
    successful_count = 0
    
    print("이슈 데이터를 가져오고 JSONL로 변환하는 중...")
    
    for i, issue_key in enumerate(issue_keys, 1):
        print(f"[{i}/{len(issue_keys)}] {issue_key} 처리 중...")
        
        
        # 임베딩용 형태로 변환
        jsonl_object = convert_to_embedding_format(issue_key)
        if jsonl_object:
            jsonl_data.append(jsonl_object)
            json_to_save_file(jsonl_object,issue_key)
            successful_count += 1
        
        # API 호출 제한 고려
        time.sleep(0.1)
    
    print(f"\n성공적으로 처리된 이슈: {successful_count}/{len(issue_keys)}")
    
    # 3. JSONL 형태로 출력
    print("\n=== JSONL 데이터 ===")
    for item in jsonl_data:
        print(json.dumps(item, ensure_ascii=False))
    
    # 4. 간단한 통계
    print(f"\n=== 통계 ===")
    print(f"총 {len(jsonl_data)}개의 JSONL 레코드 생성")
    
    # 평균 콘텐츠 길이
    if jsonl_data:
        avg_content_length = sum(len(item['content']) for item in jsonl_data) / len(jsonl_data)
        print(f"평균 콘텐츠 길이: {avg_content_length:.0f} 문자")
    
    return jsonl_data


if __name__ == "__main__":
    jsonl_results = main()

=== Jira to JSONL 변환 (임베딩용) ===
Jira URL: https://jira.suprema.co.kr
사용자: dhwoo
JQL 쿼리: project = "COMMONR" AND issuetype = "Test"

JQL로 이슈 검색을 시작합니다: project = "COMMONR" AND issuetype = "Test"
  -> 현재까지 100 / 186 개 이슈 키를 가져왔습니다...
  -> 현재까지 186 / 186 개 이슈 키를 가져왔습니다...
총 186개의 이슈를 찾았습니다.

이슈 데이터를 가져오고 JSONL로 변환하는 중...
[1/186] COMMONR-380 처리 중...
 COMMONR-380 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[2/186] COMMONR-379 처리 중...
 COMMONR-379 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[3/186] COMMONR-378 처리 중...
 COMMONR-378 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[4/186] COMMONR-377 처리 중...
 COMMONR-377 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[5/186] COMMONR-376 처리 중...
 COMMONR-376 이슈의 정보를 json으로 저장을 시작합니다.
저장 폴더: 'c:\Users\dhwoo\Desktop\project\RAG\jira_issues_output'
[6/186] COMMONR-375 처리 중...
 C